In [1]:
# Librerías:
import pandas as pd
import numpy as np

import sklearn as sk
from sklearn.metrics.pairwise import cosine_similarity
import operator
from sklearn.preprocessing import StandardScaler
import scipy.sparse as sp

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Abrir archivos para ML:
df_recom = pd.read_parquet('../Datasets/reviews.parquet')
df_games = pd.read_parquet('../Datasets/games.parquet')

In [3]:
df_recom

,user_id,item_id,recommend,review,Sentiment_analysis,Year,developer
0,76561197970982479,1250,1,Simple yet with great replayability. In my opi...,2,2011,Tripwire Interactive
1,js41637,22200,1,It's unique and worth a playthrough.,2,2011,ACE Team
2,evcentric,43110,1,Great atmosphere. The gunplay can be a bit chu...,2,2011,None
3,doctr,251610,1,I know what you think when you see this title ...,2,2014,None
4,maplemage,227300,1,For a simple (it's actually not all that simpl...,2,2013,SCS Software
...,...,...,...,...,...,...,...
25195,76561198306599751,226700,0,"This is terrible haha, even if it is on sale f...",0,2014,None
25196,Ghoustik,304930,1,Is k,1,2014,Smartly Dressed Games
25197,76561198310819422,208730,0,This game made my thighs moist. I prefer them ...,1,2014,Cyanide Studios
25198,76561198312638244,230410,1,definitely the best action game out there in t...,2,2014,Digital Extremes


In [4]:
games_item = df_games[['app_name','item_id']]
games_item.drop_duplicates(inplace=True)
games_item

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_32312\2636357517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_item.drop_duplicates(inplace=True)


,app_name,item_id
0,Lost Summoner Kitty,761140
5,Ironbound,643980
9,Real Pool 3D - Poolians,670290
14,弹炸人2222,767400
17,Log Challenge,773570
...,...,...
74823,Colony On Mars,773640
74827,LOGistICAL: South Africa,733530
74830,Russian Roads,610660
74833,EXIT 2 - Directions,658870


In [5]:
df_reviews = pd.merge(df_recom, games_item, how='left', on=['item_id'], indicator=False)
df_reviews = df_reviews[['user_id','app_name','recommend']]
df_reviews['recommend'] = df_reviews['recommend'].astype(bool)
df_reviews

,user_id,app_name,recommend
0,76561197970982479,Killing Floor,True
1,js41637,Zeno Clash,True
2,evcentric,NaN,True
3,doctr,NaN,True
4,maplemage,Euro Truck Simulator 2,True
...,...,...,...
25195,76561198306599751,NaN,False
25196,Ghoustik,Unturned,True
25197,76561198310819422,Game of Thrones,False
25198,76561198312638244,Warframe,True


In [6]:
# Obtenemos una tabla con registros únicos de cada juego:  
unique_app = df_reviews.drop_duplicates(subset="app_name")

In [7]:
# Borramos duplicados, si ya nos entregan la información necesaria:
df_sin_duplicados = df_reviews.drop_duplicates(subset=["user_id", "app_name"])

In [8]:
# Primero se comienza creando la tabla pivote user versus item_name
piv_table = df_sin_duplicados.pivot(index=["user_id"], columns=["app_name"], values="recommend").fillna(0)

In [9]:
# Verificar información dataframe:
df_sin_duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25194 entries, 0 to 25199
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    25194 non-null  object
 1   app_name   22903 non-null  object
 2   recommend  25194 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 615.1+ KB


In [10]:
# Crear la tabla pivote user_id, item_name y recommend:
piv_table = df_sin_duplicados.pivot_table(index="user_id", columns="app_name", values="recommend", fill_value=0)

In [11]:
# Se normaliza la tabla pivote con la fórmula de normalización
piv_table_norm = piv_table.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [12]:
# Se transpone la matriz, para tener juegos en filas
piv_table_norm.fillna(0, inplace=True)
piv_table_norm = piv_table_norm.T
# Se manteniene solo las columnas que tienen al menos un valor distinto de cero
piv_table_norm = piv_table_norm.loc[:, (piv_table_norm != 0).any(axis=0)]  # Tabla normal

In [13]:
piv_table_norm

user_id,--000--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-I_AM_EPIC-,-Kenny,-Mad-,-SEVEN-,-SatansLittleHelper-,-Thyme-,...,zuilde,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zy0705,zynxgameth,zyr0n1c,zzoptimuszz
app_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
//N.P.P.D. RUSH//- The milk of Ultraviolet,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
"10,000,000",-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
100% Orange Juice,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
100% Orange Juice - Krila & Kae Character Pack,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
the static speaks my name,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
theBlu,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429
theHunter Classic,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,...,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429,-0.000429


In [14]:
# Reemplazar los valores infinitos con un valor específico
piv_table_norm.replace([np.inf, -np.inf], np.nan, inplace=True)
piv_table_norm.fillna(0, inplace=True)

In [15]:
# Se crea una tabla sparse que solo guarda valores distintos de cero y permite optimizar 
piv_sparse = sp.csr_matrix(piv_table_norm.values)
piv_sparse

<2330x19985 sparse matrix of type '<class 'numpy.float64'>'
	with 46565050 stored elements in Compressed Sparse Row format>

In [16]:
# Coseno de similitud aplicado a tablas de Item (nombre de juego) y User (id de usuario)
item_simil = cosine_similarity(piv_sparse)  # Item
user_simil = cosine_similarity(piv_sparse.T) # User

In [ ]:
# Se ordenan en df
df_item_simil = pd.DataFrame(item_simil, index = piv_table_norm.index, columns = piv_table_norm.index) # Item
df_user_simil = pd.DataFrame(user_simil, index = piv_table_norm.columns, columns = piv_table_norm.columns) # User

In [ ]:
# Ver matriz de item:
df_item_simil.head(2)

item_name,"10,000,000",100% Orange Juice,1001 Spikes,1953: NATO vs Warsaw Pact,1993 Space Machine,3 Stars of Destiny,30 IMPOSSIBLE LEVELS,3SwitcheD,404Sight,7 Days to Die,...,Zombies Monsters Robots,eden*,how do you Do It?,iBomber Defense Pacific,iO,ibb & obb,sZone-Online,the static speaks my name,theHunter,Ã¤Â¾Â Ã¥Â®Â¢Ã©Â£ÂÃ¤ÂºÂÃ¤Â¼Â (Tale of Wuxia)
item_name,,,,,,,,,,,,,,,,,,,,,
"10,000,000",1.000000,0.001574,-0.009543,-0.024762,-0.016334,-0.016334,-0.020147,-0.002843,-0.004102,0.000595,...,-0.002010,-0.005810,-0.005810,0.002307,-0.020147,-0.005810,0.046021,-0.003583,-0.001215,0.002307
100% Orange Juice,0.001574,1.000000,-0.002843,-0.017860,-0.009545,-0.009545,-0.013307,0.003768,0.000593,0.005244,...,0.002664,0.000841,0.000841,0.008850,-0.013307,0.000841,0.096239,0.003038,0.002597,0.008850


In [ ]:
# Ver matriz de user:
df_user_simil.head(2)

user_id,-AnimeIsMyThing-,-GM-Dragon,1234865654,12549,1337lolroflmao,19702316748,2768820078,2d4nk4m3,2xDelorean,3456457568,...,uradumbtit,vault_brothers,wantmahbody,washington_,whodafuqisthisguilao,wirlom,xfluttersx,xoFushiox,yookobz,zomboy151
user_id,,,,,,,,,,,,,,,,,,,,,
-AnimeIsMyThing-,1.000000,-0.005533,-0.003617,-0.003617,-0.005917,-0.004673,-0.004673,0.091264,-0.002087,-0.004673,...,-0.005917,-0.002952,-0.003617,-0.004673,-0.003617,-0.002952,-0.009633,-0.003617,-0.007258,-0.004178
-GM-Dragon,-0.005533,1.000000,-0.003382,-0.003382,-0.005533,-0.004369,-0.004369,-0.007066,-0.001951,0.165411,...,-0.005533,-0.002760,0.215641,-0.004369,-0.003382,-0.002760,0.074330,-0.003382,-0.006786,-0.003907


RECOMENDACIÓN USER-ITEM:

Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [ ]:
# Los 5 juegos más recomendados similares recomendados por usuario...
def similar_user_recs(user):
    
    # Se verifica si el usuario está presente en las columnas de piv_table_norm
    if user not in df_user_simil.columns:
        return {'message': 'El Usuario no tiene datos disponibles {}'.format(user)}

    # Se obtienen los usuarios más similares 
    sim_users = df_user_simil.sort_values(by=user, ascending=False).index[1:11]

    best = []  
    most_common = {}  

    # Por cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_table_norm.loc[:, i].max()
        best.append(piv_table_norm[piv_table_norm.loc[:, i] == max_score].index.tolist())

    # Se cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1

    # Se ordenan los juegos de mayor recomendacion
    #sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    sorted_list = sorted(most_common.items(), key=lambda x: x[1], reverse=True)

    return dict(sorted_list[:5])

In [ ]:
similar_user_recs("-GM-Dragon")

{'Robocraft': 7,
 'The Mighty Quest For Epic Loot': 1,
 'theHunter': 1,
 'Counter-Strike: Global Offensive': 1,
 'March of War': 1}

In [ ]:
similar_user_recs("2d4nk4m3")

{'Unturned': 4,
 'Left 4 Dead 2 Beta': 3,
 "Garry's Mod": 1,
 'HAWKEN': 1,
 'Tropico 4': 1}

RECOMENDACIÓN ITEM-ITEM:
Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [ ]:
# Crear una tabla pivote con item_id e user_id, con las recomendaciones:
pivot_table_id = df_ML02.pivot_table(index="item_id", columns="user_id", values="recommend", fill_value=0)

In [ ]:
# Calcular un coseno de similitud:
cosine_sim = cosine_similarity(pivot_table_id)

In [ ]:
# Convertir la matriz de coseno de similitudto a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=pivot_table_id.index, columns=pivot_table_id.index)

In [ ]:
# La función para obtener el top N de items similares:
def get_similar_items(item_id, top_n=5):
    similar_items = cosine_sim_df[item_id].sort_values(ascending=False).head(top_n + 1).iloc[1:]
    return similar_items

In [ ]:
# Ejemplo: Los 5 juegos similares para item_id = 10
similar_items = get_similar_items(10)

In [ ]:
# 5 juegos similares para el id de juego 10: Juego con item_id 100, 
# Juego con item_id 80, Juego con item_id 40, Juego con item_id 240, Juego con item_id 60
similar_items 

item_id
242760    0.577350
22300     0.408248
247430    0.408248
70000     0.408248
293180    0.408248
Name: 10, dtype: float64

In [ ]:
# Guardar dataframes data_dev a parquet
piv_table_norm.to_parquet("piv_table_norm.parquet")
df_user_simil.to_parquet("df_user_simil.parquet")
cosine_sim_df.to_parquet("cosine_sim_df.parquet")